In [10]:
import numpy as np

In [31]:
#   .txt
file_name = 'test.txt'

### Deliverable 1


In [79]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

unique_char = set(content)
num_unique_char = len(unique_char)

# each character is encoded by the same number of bits
average_bit = int(np.ceil(np.log(num_unique_char)))
total_bit = average_bit * num_char

# print(content, end='\n\n')
# print(unique_char, end='\n\n')

print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
    num_char, num_unique_char, total_bit, average_bit))

Total number of characters: 201 
Number of distinct characters: 28 
Total number of bits needed to encode the text: 804 
Average number of bits per character: 4


### Deliverable 2


In [80]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

char_freq = dict()
for char in content:
    char_freq[char] = char_freq.get(char, 0) + 1

# sort unique characters by frequency in descending order
unique_char_sorted = sorted(
    char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

total_bit = 0
i = 1
for char in unique_char_sorted:
    total_bit += char_freq[char] * i
    i += 1

average_bit = total_bit / num_char

print('Total numer of bits: {} \nAverage number of bits: {}'.format(
    total_bit, average_bit))

Total numer of bits: 1672 
Average number of bits: 8.318407960199005


### Deliverable 3


In [75]:
class Node:
    def __init__(self, chars, freq, left=None, right=None):
        self.chars = chars
        self.freq = freq
        self.left = left
        self.right = right

In [1]:
def get_char_freq(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    return char_freq

In [ ]:
def create_binary_tree(char_freq):
    # sort unique characters by frequency in descending order
    unique_char_sorted = sorted(
        char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

    root = Node(char_freq.keys(), sum(char_freq.values()))

    for char in unique_char_sorted:
        root.left = Node([char], char_freq[char])
        root.right = Node(root.chars.remove(char), root.freq - char_freq[char])

        root = root.right
        if len(root.chars) == 1:
            break

    return root

In [ ]:
def merge_tree(root_1, root_2):
    chars = list(set(root_1.chars + root_2.chars))
    freq = root_1.freq + root_2.freq

    return Node(chars, freq, root_1, root_2)